# Multiple Linear Regression Model
## Part C - Section IV

This part contains the linear mixed model performed in R.

Requires library nlme from R.

---
**Description:** reproduction of the results presented in section IV.B in the paper 'Online Sharing of physical activity'.

**Author:** Eric Araujo

---

In [1]:
# Compute random effects model and compare it with gls (generalized least square) 
# The first week is left out, because this week is usually a bit atypical,
# presumably due to novelty effects of starting the program.
library(nlme)

In [2]:
getWholeCommunity <- function(){
  path <- "./data/original"
    # "D:/research/directLifeData/DirectLife/Analysis.0.4/results/statistical analysis"
  communityWithoutNA <- read.csv(paste(path, "/allCommunitiesWithoutNA.csv", sep=""))
  return (communityWithoutNA)
}

In [3]:
getWholeNonCommunity <- function(){
  path <- "./data/original"
  nonCommunityWithoutNA <- read.csv(paste(path, "/allnonCommunitiesWithoutNA.csv", sep=""))
  return (nonCommunityWithoutNA)
}

In [4]:
community <- getWholeCommunity()
nonCommunity <- getWholeNonCommunity()

In [5]:
drops  <- c("X.1", "X.2", "X.3", "X.4", "X.5", "X.6", "X.7") 
#first week is removed because of the novelity effect(see paper)

In [6]:
communityLongFormat <- community[, !(names(community) %in% drops)]
nonCommunityLongFormat <- nonCommunity[, !(names(nonCommunity) %in% drops)]

In [7]:
# reshape function converts it to long format
nonCommunityLongFormat <- reshape(nonCommunityLongFormat, varying = 2:78, idvar = "id", direction = "long")
communityLongFormat <- reshape(communityLongFormat, varying = 2:78, idvar = "id", direction = "long")

In [8]:
nonCommunityLongFormat["isCommunity"] <- as.factor(0)
communityLongFormat["isCommunity"] <- as.factor(1)

In [9]:
communityNoncommunityLongFormat <- rbind(communityLongFormat, nonCommunityLongFormat)
communityNoncommunityLongFormat <- within(communityNoncommunityLongFormat, communityNoncommunityLongFormat$isCommunity <- relevel(communityNoncommunityLongFormat$isCommunity, ref = 1))
model1 <- lme(X ~ isCommunity*time, random = ~1|id, data = communityNoncommunityLongFormat)
model2 <- gls(X ~ isCommunity*time, data = communityNoncommunityLongFormat)

In [10]:
print ("***************************************************************")
print("M O D E L 1: L M E")
print ("***************************************************************")
print (summary(model1))
print ("###############################################################")

[1] "***************************************************************"
[1] "M O D E L 1: L M E"
[1] "***************************************************************"
Linear mixed-effects model fit by REML
 Data: communityNoncommunityLongFormat 
       AIC      BIC    logLik
  13555.15 13607.41 -6771.574

Random effects:
 Formula: ~1 | id
        (Intercept)  Residual
StdDev:   0.2067486 0.2744509

Fixed effects: X ~ isCommunity * time 
                       Value   Std.Error    DF  t-value p-value
(Intercept)        1.6933389 0.023882981 44230 70.90149  0.0000
isCommunity0      -0.0586920 0.025818758   580 -2.27323  0.0234
time               0.0005821 0.000153538 44230  3.79115  0.0002
isCommunity0:time -0.0006525 0.000165983 44230 -3.93109  0.0001
 Correlation: 
                  (Intr) isCmm0 time  
isCommunity0      -0.925              
time              -0.296  0.274       
isCommunity0:time  0.274 -0.296 -0.925

Standardized Within-Group Residuals:
        Min          Q1         

In [11]:
print ("***************************************************************")
print("M O D E L 2: G L S")
print ("***************************************************************")
print (summary(model2))
print ("###############################################################")

print ("***************************************************************")
print ("comparing simple model with random model")
print ("***************************************************************")

[1] "***************************************************************"
[1] "M O D E L 2: G L S"
[1] "***************************************************************"
Generalized least squares fit by REML
  Model: X ~ isCommunity * time 
  Data: communityNoncommunityLongFormat 
       AIC      BIC    logLik
  31435.78 31479.33 -15712.89

Coefficients:
                       Value   Std.Error   t-value p-value
(Intercept)        1.6933389 0.009814656 172.53167  0.0000
isCommunity0      -0.0586920 0.010610159  -5.53168  0.0000
time               0.0005821 0.000192112   3.02993  0.0024
isCommunity0:time -0.0006525 0.000207683  -3.14177  0.0017

 Correlation: 
                  (Intr) isCmm0 time  
isCommunity0      -0.925              
time              -0.900  0.833       
isCommunity0:time  0.833 -0.900 -0.925

Standardized residuals:
        Min          Q1         Med          Q3         Max 
-1.83791425 -0.53524047 -0.08126548  0.45666652 63.72657283 

Residual standard error: 0.343402

In [12]:
result <- anova(model1, model2)
print (result)

       Model df      AIC      BIC     logLik   Test  L.Ratio p-value
model1     1  6 13555.15 13607.41  -6771.574                        
model2     2  5 31435.78 31479.33 -15712.890 1 vs 2 17882.63  <.0001
